In [2]:
def kyle_print(string):
    print(string)

In [3]:
import matplotlib as mpl
from PIL import ImageColor

#Alex's code.
def color_fader(cs, mix=0):
    if len(cs) == 2:
        c1 = np.array(mpl.colors.to_rgb(cs[0]))
        c2 = np.array(mpl.colors.to_rgb(cs[1]))
        return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
    elif len(cs) == 3:
        c1 = np.array(mpl.colors.to_rgb(cs[0]))
        c2 = np.array(mpl.colors.to_rgb(cs[1]))
        c3 = np.array(mpl.colors.to_rgb(cs[2]))
        if mix <= .5:
            mix *= 2 # create 0-1 range
            return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
        else:
            mix -= .5 # create 0-1 range
            mix *= 2
            return mpl.colors.to_hex((1-mix)*c2 + mix*c3)

def create_color_buckets(colors=['#FF0000', '#00FF00'], num_buckets=10, display=True): #RGB
    color_buckets = []

    if display:
        fig, ax = plt.subplots(figsize=(4, 2))
    
    for x in range(0, num_buckets):
        X = color_fader(colors, x/(num_buckets-1))
        
        if display:
            ax.axvline(x, color=X, linewidth=4)
        
        color_buckets.append(ImageColor.getcolor(X, "RGB")[::-1]) # convert RGB to BGR
        
    if display:
        plt.axis('off')
        plt.show()

    return color_buckets

In [8]:
import copy
import cv2
import imutils
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import ImageColor
from skimage import exposure
from skimage.measure import label, regionprops
from skimage.morphology import area_opening, disk, square
import sys

def scoregrass(image1, sensitivity_list):  
    hsi_images1 = []
    hsv1 = cv2.cvtColor(image1, cv2.COLOR_BGR2HSV)
    hsi_images1.append(hsv1)
    imgreturn = hsv1
    hue_template = np.zeros(hsv1.shape[0:2])
    
    for sensitivity in sensitivity_list:
        #where is hue within 60+-sensitivity
        lower = np.array([60 - sensitivity]).astype(np.uint8)
        upper = np.array([60+sensitivity]).astype(np.uint8)
        mask = cv2.inRange(hsv1[:,:,0], lower, upper).astype(bool)
        hue_template[mask] = hue_template[mask]+1
        '''for i in range(0, hsv1.shape[0]): #iterate over y position
            for j in range(0, hsv1.shape[1]): # iterate over x position   
                hue = hsv1[i][j][0]
                if(hue in range(60-sensitivity,60+sensitivity)): #look at this 60 value in more detail.
                    score_return[i][j] = score_return[i][j] + 1
                imgreturn = hsv1'''
        imgreturn = hsv1
    return imgreturn,hue_template      
    # define range of blue color in HSV
    #lower_green = np.array([60 - sensitivity, 100, 50])
    #upper_green = np.array([60 + sensitivity, 255, 255])
    # Threshold the HSV image to get only blue colors
    #mask1 = cv2.inRange(hsv1, lower_green, upper_green)
    # Bitwise-AND mask and original image
    #res1 = cv2.bitwise_and(image1,image1, mask= mask1)
    #cv2.imshow('image1',image1)
    #cv2.imshow('mask1',mask1)
    #cv2.imshow('res1',res1)
    


'''def absolute_image_differencing_with_HSL(image1,image2,threshold):
    #First convert the image to HSL
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2HLS)
    image2 = cv2.cvtColor(image2,cv2.COLOR_BGR2HLS)
    print(image1)
    #get only the hue channel
    image1hues = image1[:,:,0]
    image2hues = image2[:,:,0]
    for each in image1hues:
        print(each)
    return 0

image_differences = []
'''



In [12]:
def setup_buckets(num_buckets_val):
    color_buckets = create_color_buckets(colors=['#FF0000', '#00FF00'], num_buckets=num_buckets_val, display=True)
    color_buckets.insert(0,(0,0,0))
    return color_buckets

def score_missions(regions,num_buckets_val):
    #regions are just the grass regions.
    total_grass_region_pixels = np.sum(regions[0] != [0,0,0]) #don't include black pixels.
    color_buckets = setup_buckets(num_buckets_val)
    grass_differences = []
    percent_buckets = [] #list of lists. #return the percentages for every image that are in each color.
    for i in range(0,len(regions)):#score every mission image
        current_bucket_percent_list = []
        image1 = copy.deepcopy(regions[i])
        #score_return = np.zeros((image1.shape[0:2])) #creates
            #rad1=5 
            #rad2=10
            #rad3=17
        linspace_values = np.linspace(15, 37, num_buckets_val-1).astype(int)
        score_return = scoregrass(image1,linspace_values)[1] #try num_buckets different sensitivities.
            #score_return2 = scoregrass(image1,rad1)
        var = copy.deepcopy(score_return)
        #var = var*25  #use this is we're using grayscale to better visualize changes between black and white
        color_score = np.zeros((image1.shape))
        for score in range(0, num_buckets_val):
            mask = score_return == score
            color_score[mask, :] = color_buckets[score]
            val = np.multiply(np.divide(np.sum(mask),total_grass_region_pixels),100)
            current_bucket_percent_list.append(val)  
        variable = copy.deepcopy(color_score)
        grass_differences.append(variable.astype(np.uint8))        #print(i)             #imgDifference = absolute_image_differencing_with_HSL(image1,image2,25)             #image_differences.append(imgDifference)
        percent_buckets.append(current_bucket_percent_list)
    return grass_differences, percent_buckets